In [ ]:
!pip install transformers datasets sentence-transformers torch accelerate bitsandbytes konlpy dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
from sentence_transformers import SentenceTransformer
# from tensorflow.keras.datasets import imdb
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from google.colab import drive
drive.mount('/content/drive')
import os
import json
colab_path = "/content/drive/MyDrive/Colab Notebooks/KUBIG-25S"
os.chdir(colab_path)

Mounted at /content/drive


# 1. Translation

In [ ]:
client_id = ""
client_secret = ""

def papago_translate(text, source='en', target='ko'):
    url = "https://papago.apigw.ntruss.com/nmt/v1/translation"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    data = {
        "source": source,
        "target": target,
        "text": text
    }
    response = requests.post(url, headers=headers, data=data)
    return response.json()

def prepare_data(start=0, end=5000, max_length = 2000):

    (indice_en, labels), (x_test, y_test) = imdb.load_data()
    indice_en = np.concatenate([indice_en, x_test])[start:end]
    labels_all = np.concatenate([labels, y_test])[start:end]


    word_index = imdb.get_word_index()
    reverse_index = {value + 3: key for key, value in word_index.items()}
    reverse_index[0] = "<PAD>"
    reverse_index[1] = ""
    reverse_index[2] = "<UNK>"
    reverse_index[10] = "\n"

    texts_en = []
    labels = []
    for idx, label in zip(indice_en, labels_all):
        text_en = ' '.join([reverse_index.get(i, '?') for i in idx]).strip()
        if len(text_en) < max_length:
            texts_en.append(text_en)
            labels.append(label)

    print("Translating English to Korean ...")

    texts_ko = []
    valid_texts_en = []
    valid_labels = []

    for text, label in tqdm(zip(texts_en, labels), total=len(texts_en)):
        try:
            result = papago_translate(text)
            translated = result['message']['result']['translatedText']
            texts_ko.append(translated)
            valid_texts_en.append(text)
            valid_labels.append(label)
        except Exception as e:
            print(f"[Error] Skipping due to: {e}")

    return pd.DataFrame({"en": valid_texts_en, "ko": texts_ko, "label": valid_labels})

test = prepare_data()

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Translating English to Korean ...


100%|██████████| 77/77 [01:41<00:00,  1.32s/it]


In [ ]:
test.to_csv("/content/drive/MyDrive/Colab Notebooks/translated_imdb_sample.csv",index=False)

test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/translated_imdb_sample.csv")

# 2. Adversarial Learning(번역IMDB-쇼핑리뷰)
1. 한국어 번역 데이터(소스)와 한국어 데이터(타겟)으로 embedding과 classifier를 한번에 학습(klue bert)
2. classification loss + MMD loss(domain loss)

## Dataset

### IMDB(20K; Translated)

In [ ]:
imdb_df = pd.read_csv("translated_imdb.csv").iloc[:20000,:]

imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19525 entries, 0 to 19524
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   en      19525 non-null  object
 1   ko      19525 non-null  object
 2   label   19525 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 457.7+ KB


### Naver Review(180K + 5K; +Preprocessing)

In [ ]:
import re

naver = pd.read_csv("Train_target.csv").iloc[:20000,:]
naver_val = pd.read_csv("Valid_target.csv").iloc[:5000,:]

slang_dict1 = {
    "ㅈㄴ": "너무",
    "ㄹㅇ": "정말로",
    "ㅂㅂ": "안녕",
    "ㅇㅈ": "인정",
    "ㄴㄴ": "금지",
    "ㅋㅋ": "키키",
    "ㅋㅎ": "키키",
    "ㅎㅎ": "히히",
    "ㅎㅋ": "히히",
    "ㅜㅜ": "흑흑",
    "ㅜㅠ": "흑흑",
    "ㅠㅠ": "흑흑",
    "ㅠㅜ": "흑흑",
    "ㅡㅡ": "실망",
    "ㅅㅂ": "최악",
    "ㅆㅂ": "최악",
    "ㅆㅃ": "최악",
    "ㅄ": "최악",
    "ㅂㅅ": "최악",
    "ㅈㄹ": "최악"
    # 추가적인 신조어 및 비속어를 여기에 계속 추가
}

slang_dict2 = {
    "ㅋ": "킥",
    "ㅎ": "헤헤",
    "ㅜ": "으앙",
    "ㅜ": "으앙"
    # 추가적인 신조어 및 비속어를 여기에 계속 추가
}

def preprocessing(text):
    for k, v in slang_dict1.items():
            text = text.replace(k, v)

    for k, v in slang_dict2.items():
            text = text.replace(k, v)

    return re.sub(r"(.)\1{2,}", r"\1\1", text) # 반복표현 제거

naver['text'] = naver['text'].apply(preprocessing)
naver_val['text'] = naver_val['text'].apply(preprocessing)

## Train

### DataLoader

In [ ]:
class DomainDataset(Dataset):
    def __init__(self, texts, domain_labels, tokenizer, max_len=512, sentiment_labels=None):
        self.texts = texts
        self.domain_labels = domain_labels
        self.sentiment_labels = sentiment_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt")
        item = {key: val.squeeze(0) for key, val in encoded.items()}
        item['domain_label'] = torch.tensor(self.domain_labels[idx])
        item['sentiment_label'] = torch.tensor(int(self.sentiment_labels[idx]), dtype=torch.long)
        return item

src_texts = imdb_df['ko'].tolist()
tgt_texts = naver['text'].tolist()
src_labels = imdb_df['label'].tolist()

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

src_ds = DomainDataset(src_texts, [0] * len(src_texts), tokenizer, sentiment_labels=src_labels)
tgt_ds = DomainDataset(tgt_texts, [1] * len(tgt_texts), tokenizer, sentiment_labels=[-1] * len(tgt_texts))
val_ds = DomainDataset(naver_val['text'].tolist(), [1] * len(naver_val), tokenizer, sentiment_labels=naver_val['label'].tolist())

src_loader = DataLoader(src_ds, batch_size=32, shuffle=True, drop_last=True)
tgt_loader = DataLoader(tgt_ds, batch_size=32, shuffle=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, drop_last=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

### Model

In [ ]:
import torch.nn.functional as F

def compute_mmd(x_src, x_tgt, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = x_src.size(0)
    x = torch.cat([x_src, x_tgt], dim=0)

    # Compute pairwise squared distances
    xx = torch.matmul(x, x.t())
    x2 = torch.sum(x * x, dim=1, keepdim=True)
    squared_dist = x2 + x2.t() - 2 * xx

    # Compute kernels
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(squared_dist.detach()) / (batch_size ** 2 - batch_size)

    bandwidth_list = [bandwidth * (kernel_mul ** i) for i in range(kernel_num)]
    kernels = [torch.exp(-squared_dist / bw) for bw in bandwidth_list]
    kernel_matrix = sum(kernels)

    # Split kernel matrix
    K_XX = kernel_matrix[:batch_size, :batch_size]
    K_YY = kernel_matrix[batch_size:, batch_size:]
    K_XY = kernel_matrix[:batch_size, batch_size:]
    K_YX = kernel_matrix[batch_size:, :batch_size]

    mmd = torch.mean(K_XX + K_YY - K_XY - K_YX)
    return mmd

class MMDModel(nn.Module):
    def __init__(self,hidden_dim=128):
        super().__init__()
        self.bert = AutoModel.from_pretrained("klue/bert-base")
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls = outputs.last_hidden_state[:, 0]  # CLS 토큰
        sentiment_logits = self.classifier(cls)
        return sentiment_logits, cls

> 손실 계수 튜닝 및 시각화

In [ ]:
def train_mmd_model(src_loader, tgt_loader, val_loader, num_epochs=5, beta=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MMDModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    sentiment_criterion = nn.CrossEntropyLoss(ignore_index=-1)

    best_val_loss = float("inf")
    best_model_state = None
    save_path = f"models/mmd_model_beta{beta}.pth"

    for epoch in range(num_epochs):
        model.train()
        total_cls_loss, total_mmd_loss = 0.0, 0.0

        src_iter = iter(src_loader)
        tgt_iter = iter(tgt_loader)
        steps_per_epoch = min(len(src_loader), len(tgt_loader))

        for step in tqdm(range(steps_per_epoch), desc=f"Epoch {epoch+1} - Train"):
            try:
                src_batch = next(src_iter)
                tgt_batch = next(tgt_iter)
            except StopIteration:
                continue

            input_ids_src = src_batch['input_ids'].to(device)
            mask_src = src_batch['attention_mask'].to(device)
            labels_src = src_batch['sentiment_label'].to(device)

            input_ids_tgt = tgt_batch['input_ids'].to(device)
            mask_tgt = tgt_batch['attention_mask'].to(device)

            logits_src, feat_src = model(input_ids_src, mask_src)
            _, feat_tgt = model(input_ids_tgt, mask_tgt)

            cls_loss = sentiment_criterion(logits_src, labels_src)
            mmd_loss = compute_mmd(feat_src, feat_tgt)

            loss = cls_loss + beta * mmd_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_cls_loss += cls_loss.item()
            total_mmd_loss += mmd_loss.item()

        print(f"[Epoch {epoch+1}] Train Sentiment Loss: {total_cls_loss/steps_per_epoch:.4f} | MMD Loss: {total_mmd_loss/steps_per_epoch:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0
        correct, total = 0, 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} - Valid"):
                input_ids = batch['input_ids'].to(device)
                mask = batch['attention_mask'].to(device)
                labels = batch['sentiment_label'].to(device)

                logits, _ = model(input_ids, mask)
                loss = sentiment_criterion(logits, labels)
                val_loss += loss.item()

                preds = logits.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"[Epoch {epoch+1}] Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            torch.save(best_model_state, save_path)
            print(f">> Saved best model at epoch {epoch+1}")

        # visualization
        feats_all, domain_markers, label_colors = [], [], []
        model.eval()
        src_iter_vis = iter(src_loader)
        tgt_iter_vis = iter(val_loader)
        steps = 1000 // src_loader.batch_size

        with torch.no_grad():
            for _ in range(steps):
                try:
                    src_batch = next(src_iter_vis)
                    tgt_batch = next(tgt_iter_vis)
                except StopIteration:
                    break

                # Source
                input_ids_src = src_batch['input_ids'].to(device)
                mask_src = src_batch['attention_mask'].to(device)
                labels_src = src_batch['sentiment_label'].cpu().numpy()
                _, feat_src = model(input_ids_src, mask_src)

                feats_all.append(feat_src.cpu().numpy())
                domain_markers += ['source'] * len(feat_src)
                label_colors += list(labels_src)

                # Target
                input_ids_tgt = tgt_batch['input_ids'].to(device)
                mask_tgt = tgt_batch['attention_mask'].to(device)
                labels_tgt = tgt_batch['sentiment_label'].cpu().numpy()
                _, feat_tgt = model(input_ids_tgt, mask_tgt)

                feats_all.append(feat_tgt.cpu().numpy())
                domain_markers += ['target'] * len(feat_tgt)
                label_colors += list(labels_tgt)

        feats_all = np.vstack(feats_all)
        domain_markers = np.array(domain_markers)
        label_colors = np.array(label_colors)

        tsne = TSNE(n_components=2, perplexity=30, random_state=42, init='pca', learning_rate='auto')
        feats_2d = tsne.fit_transform(feats_all)

        plt.figure(figsize=(8, 6))
        classes = np.unique(label_colors)
        markers = {'source': 'o', 'target': 'x'}
        cmap = plt.get_cmap("tab10", 10*len(classes))

        for cls in classes:
            for domain in ['source', 'target']:
                idxs = (label_colors == cls) & (domain_markers == domain)
                plt.scatter(
                    feats_2d[idxs, 0], feats_2d[idxs, 1],
                    c=[cmap(5*cls)], marker=markers[domain],
                    label=f"{domain}-class{cls}", alpha=0.6
                )

        plt.legend()
        plt.title(f"t-SNE Embedding Visualization (beta {beta}, Epoch {epoch+1})")
        plt.tight_layout()
        plt.savefig(f"visualizations/adversarial_beta{beta}_epoch_{epoch+1}_tsne.png")
        plt.close()

    model.load_state_dict(best_model_state)
    return model


In [ ]:
def visualize_initial_embeddings(model, src_loader, tgt_loader, beta=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    feats_all, domain_markers, label_colors = [], [], []
    src_iter = iter(src_loader)
    tgt_iter = iter(val_loader)
    steps = 1000 // src_loader.batch_size  # 또는 원하는 시각화 샘플 수로 조정

    with torch.no_grad():
        for _ in range(steps):
            try:
                src_batch = next(src_iter)
                tgt_batch = next(tgt_iter)
            except StopIteration:
                break

            # Source
            input_ids_src = src_batch['input_ids'].to(device)
            mask_src = src_batch['attention_mask'].to(device)
            labels_src = src_batch['sentiment_label'].cpu().numpy()
            _, feat_src = model(input_ids_src, mask_src)

            feats_all.append(feat_src.cpu().numpy())
            domain_markers += ['source'] * len(feat_src)
            label_colors += list(labels_src)

            # Target
            input_ids_tgt = tgt_batch['input_ids'].to(device)
            mask_tgt = tgt_batch['attention_mask'].to(device)
            labels_tgt = tgt_batch['sentiment_label'].cpu().numpy()
            _, feat_tgt = model(input_ids_tgt, mask_tgt)

            feats_all.append(feat_tgt.cpu().numpy())
            domain_markers += ['target'] * len(feat_tgt)
            label_colors += list(labels_tgt)

    # Combine and visualize
    feats_all = np.vstack(feats_all)
    domain_markers = np.array(domain_markers)
    label_colors = np.array(label_colors)

    tsne = TSNE(n_components=2, perplexity=30, random_state=42, init='pca', learning_rate='auto')
    feats_2d = tsne.fit_transform(feats_all)

    plt.figure(figsize=(8, 6))
    classes = np.unique(label_colors)
    markers = {'source': 'o', 'target': 'x'}
    cmap = plt.get_cmap("tab10", 10*len(classes))

    for cls in classes:
        for domain in ['source', 'target']:
            idxs = (label_colors == cls) & (domain_markers == domain)
            plt.scatter(
                feats_2d[idxs, 0], feats_2d[idxs, 1],
                c=[cmap(5*cls)], marker=markers[domain],
                label=f"{domain}-class{cls}", alpha=0.6
            )

    plt.legend()
    plt.title(f"t-SNE Embedding Visualization (Initial, beta {beta})")
    plt.tight_layout()
    plt.savefig(f"visualizations/adversarial_beta{beta}_epoch_0_tsne.png")
    plt.close()

In [ ]:
model = MMDModel()
visualize_initial_embeddings(model, src_loader, tgt_loader, beta=1.0)

In [ ]:
train_mmd_model(src_loader, tgt_loader, val_loader, beta=1.0)
torch.cuda.empty_cache()

Epoch 1 - Train: 100%|██████████| 610/610 [12:06<00:00,  1.19s/it]


[Epoch 1] Train Sentiment Loss: 0.3045 | MMD Loss: 0.0329


Epoch 1 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.06it/s]


[Epoch 1] Val Loss: 0.2574, Accuracy: 0.8998
>> Saved best model at epoch 1


Epoch 2 - Train: 100%|██████████| 610/610 [12:06<00:00,  1.19s/it]


...

[Epoch 4] Val Loss: 0.4319, Accuracy: 0.8844


Epoch 5 - Train: 100%|██████████| 610/610 [12:05<00:00,  1.19s/it]


[Epoch 5] Train Sentiment Loss: 0.0437 | MMD Loss: 0.0284


Epoch 5 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.06it/s]


[Epoch 5] Val Loss: 0.3589, Accuracy: 0.8980


In [ ]:
train_mmd_model(src_loader, tgt_loader, val_loader, beta=0.8)
torch.cuda.empty_cache()

Epoch 1 - Train: 100%|██████████| 610/610 [12:06<00:00,  1.19s/it]


[Epoch 1] Train Sentiment Loss: 0.3123 | MMD Loss: 0.0321


Epoch 1 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.05it/s]


[Epoch 1] Val Loss: 0.2639, Accuracy: 0.9030
>> Saved best model at epoch 1


Epoch 2 - Train: 100%|██████████| 610/610 [12:06<00:00,  1.19s/it]


...

[Epoch 4] Val Loss: 0.3830, Accuracy: 0.8964


Epoch 5 - Train: 100%|██████████| 610/610 [12:06<00:00,  1.19s/it]


[Epoch 5] Train Sentiment Loss: 0.0375 | MMD Loss: 0.0280


Epoch 5 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.04it/s]


[Epoch 5] Val Loss: 0.3674, Accuracy: 0.8936


In [ ]:
train_mmd_model(src_loader, tgt_loader, val_loader, beta=0.6)
torch.cuda.empty_cache()

Epoch 1 - Train: 100%|██████████| 610/610 [12:07<00:00,  1.19s/it]


[Epoch 1] Train Sentiment Loss: 0.2995 | MMD Loss: 0.0341


Epoch 1 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.03it/s]


[Epoch 1] Val Loss: 0.2651, Accuracy: 0.9006
>> Saved best model at epoch 1


Epoch 2 - Train: 100%|██████████| 610/610 [12:07<00:00,  1.19s/it]


...

[Epoch 4] Val Loss: 0.3579, Accuracy: 0.9042


Epoch 5 - Train: 100%|██████████| 610/610 [12:07<00:00,  1.19s/it]


[Epoch 5] Train Sentiment Loss: 0.0436 | MMD Loss: 0.0275


Epoch 5 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.04it/s]


[Epoch 5] Val Loss: 0.3886, Accuracy: 0.9034


In [ ]:
train_mmd_model(src_loader, tgt_loader, val_loader, beta=1.2)
torch.cuda.empty_cache()

Epoch 1 - Train: 100%|██████████| 610/610 [12:07<00:00,  1.19s/it]


[Epoch 1] Train Sentiment Loss: 0.3245 | MMD Loss: 0.0340


Epoch 1 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.03it/s]


[Epoch 1] Val Loss: 0.2524, Accuracy: 0.9078
>> Saved best model at epoch 1


Epoch 2 - Train: 100%|██████████| 610/610 [12:07<00:00,  1.19s/it]


...

[Epoch 4] Val Loss: 0.2975, Accuracy: 0.9044


Epoch 5 - Train: 100%|██████████| 610/610 [12:07<00:00,  1.19s/it]


[Epoch 5] Train Sentiment Loss: 0.0416 | MMD Loss: 0.0288


Epoch 5 - Valid: 100%|██████████| 157/157 [00:31<00:00,  5.03it/s]


[Epoch 5] Val Loss: 0.3798, Accuracy: 0.8992
